In [2]:
import pandas as pd

In [16]:
df = pd.read_csv('../data/test_all_steps.csv')

In [17]:
df.clone_id.unique()

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [20]:
df.groupby('clone_id')['step'].count()

clone_id
-1     294979
 0        470
 1        424
 2        345
 3        236
 4        200
 5        547
 6        308
 7        656
 8        200
 9        562
 10       286
 11       200
 12       620
 13       420
 14       317
 15       200
Name: step, dtype: int64